In [1]:
#Loading Data. Will take some time
import pandas as pd
import os
import glob
import collections
import math
import re
data = []
data1 = glob.glob(r'IMDB_Dataset\Dataset\train\pos\*.txt')
data2 = glob.glob(r'IMDB_Dataset\Dataset\train\neg\*.txt')
data3 = glob.glob(r'IMDB_Dataset\Dataset\test\pos\*.txt')
data4 = glob.glob(r'IMDB_Dataset\Dataset\test\neg\*.txt')
postrain_X = []
negtrain_X = []
postest_X = []
negtest_X = []

for i in range(len(data1)):
    file = open(data1[i], 'r', encoding="utf8")
    postrain_X.append(file.read().replace('\n', ''))        
for i in range(len(data2)):
    file = open(data2[i], 'r', encoding="utf8")
    negtrain_X.append(file.read().replace('\n', ''))
for i in range (len(data3)):
    file = open(data3[i], 'r', encoding="utf8")
    postest_X.append(file.read().replace('\n', ''))
for i in range(len(data4)):
    file = open(data4[i], 'r', encoding="utf8")
    negtest_X.append(file.read().replace('\n', ''))    

In [2]:
#Pre-processing. Takes quite a while
ppostrain_X = [] #pre-processed training data with positive class
pnegtrain_X = []
ppostest_X = []
pnegtest_X = []
file = open('IMDB_Dataset\Dataset\stop_words.txt', 'r')
lines = file.readlines()
for i in range(0, len(postrain_X)):
    ppostrain_X.append(postrain_X[i].lower())
    ppostrain_X[i] = re.sub(r"[.,;:!?0-9]", "" ,ppostrain_X[i])
    ppostrain_X[i] = re.sub(r"[^a-z\s']", "", ppostrain_X[i])

    pnegtrain_X.append(negtrain_X[i].lower())
    pnegtrain_X[i] = re.sub(r"[.,;:!?0-9]", "" ,pnegtrain_X[i])
    pnegtrain_X[i] = re.sub(r"[^a-z\s']", "", pnegtrain_X[i])

    ppostest_X.append(postest_X[i].lower())
    ppostest_X[i] = re.sub(r"[.,;:!?0-9]", "" ,ppostest_X[i])
    ppostest_X[i] = re.sub(r"[^a-z\s']", "", ppostest_X[i])

    pnegtest_X.append(negtest_X[i].lower())
    pnegtest_X[i] = re.sub(r"[.,;:!?0-9]", "" ,pnegtest_X[i])
    pnegtest_X[i] = re.sub(r"[^a-z\s']", "", pnegtest_X[i])

    for line in lines:
        stop_word = line.strip()
        ppostrain_X[i] = re.sub(r"\b{}\b".format(stop_word), "" ,ppostrain_X[i])
        pnegtrain_X[i] = re.sub(r"\b{}\b".format(stop_word), "" ,pnegtrain_X[i])
        ppostest_X[i] = re.sub(r"\b{}\b".format(stop_word), "" ,ppostest_X[i])
        pnegtest_X[i] = re.sub(r"\b{}\b".format(stop_word), "" ,pnegtest_X[i])
    ppostrain_X[i] = re.sub(r"[']", "", ppostrain_X[i])
    pnegtrain_X[i] = re.sub(r"[']", "", pnegtrain_X[i])
    ppostest_X[i] = re.sub(r"[']", "", ppostest_X[i])
    pnegtest_X[i] = re.sub(r"[']", "", pnegtest_X[i])

In [3]:
def make_vocabulary(trd):
    vocabulary = set()
    for i in trd:
        words = i.split()
        for j in words:
            vocabulary.add(j)
    return vocabulary

In [4]:
def count_words(words):
    dict1 = {}
    for word in words:
        if(word in dict1.keys()):
            dict1[word] = dict1[word] + 1
        else:
            dict1[word] = 1
    return dict1

In [5]:
def make_bow(trd, vocabulary):
    bow = {}
    big_string = ' '.join(trd)
    words = big_string.split()
    frequency = count_words(words)
    for j in range(len(vocabulary)):
        bow[vocabulary[j]] = 0
        if vocabulary[j] in frequency.keys():
            bow[vocabulary[j]] = frequency[vocabulary[j]]
    return bow

In [6]:
#Part1
def train_naive_bayes(train_X, C, vocabulary): #vocabulary given as parameter since there is no need to find vocabulary in loop again and again since it wont change.
    logprior = {}
    bigdoc = {}
    loglikelihood = {}
    for c in C.keys():
        Ndoc = len(train_X)
        Nc = len(C[c])
        total_count = 0
        bigdoc[c] = make_bow(C[c], vocabulary)
        for word in bigdoc[c].keys(): #added +1 to count of each word
            total_count = total_count + 1 + bigdoc[c][word] 
        logprior[c] = math.log(Nc/Ndoc)
        dict_words = {}
        for word in bigdoc[c].keys():
            count = bigdoc[c][word]
            dict_words[word] = math.log((count + 1)/total_count)
        loglikelihood[c] = dict_words    
    return logprior, loglikelihood    

In [7]:
def test_naive_bayes(testdoc, logprior, loglikelihood, C, bigdoc):
    sums = {}
    words = testdoc.split()
    for c in C.keys():
        sums[c] = logprior[c]
        for word in words:
            if word in bigdoc[c].keys():
                sums[c] = sums[c] + loglikelihood[c][word]
    return (max(sums, key = sums.get))  

In [8]:
def confusion_matrix(true_pos, false_pos, true_neg, false_neg):
    confusion_matrix = np.zeros((2, 2), dtype = int)
    confusion_matrix[0][0] = true_neg
    confusion_matrix[0][1] = false_pos
    confusion_matrix[1][0] = false_neg
    confusion_matrix[1][1] = true_pos
    return confusion_matrix

In [9]:
train_X = ppostrain_X + pnegtrain_X #training data with positive labels and training data with negative lables combined together
test_X = ppostest_X + pnegtest_X
vocabulary = list(make_vocabulary(train_X))
C = {}
C['pos'] = ppostrain_X
C['neg'] = pnegtrain_X
logprior, loglikelihood = train_naive_bayes(train_X, C, vocabulary)

In [10]:
bigdoc = {}
test_pred = []
bigdoc['pos'] = make_bow(C['pos'], vocabulary) # will be passing bigdoc as argument instead of vocabulary as vocabulary is a list and finidng the word in list takes much more time than finding word in dictionary.keys()
bigdoc['neg'] = make_bow(C['neg'], vocabulary)
for review in test_X:
    test_pred.append(test_naive_bayes(review, logprior, loglikelihood, C, bigdoc))

In [11]:
import numpy as np
true_pos = 0
true_neg = 0
false_pos = 0
false_neg = 0
for i in range(len(ppostest_X)):
    if(test_pred[i] == 'pos'):
        true_pos+=1
    else:
        false_neg+=1
for i in range(len(ppostest_X), len(ppostest_X) + len(pnegtest_X)):
    if(test_pred[i] == 'neg'):
        true_neg+=1
    else:
        false_pos+=1
accuracy = (true_pos + true_neg) / (true_pos + true_neg + false_pos + false_neg)
print('Accuracy is: ', accuracy)
print('Confusion matrix is: \n')
print(confusion_matrix(true_pos, false_pos, true_neg, false_neg))


Accuracy is:  0.82644
Confusion matrix is: 

[[11001  1499]
 [ 2840  9660]]


In [12]:
#Part2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
vectorizer = CountVectorizer()
X1 = vectorizer.fit_transform(train_X)
X2 = vectorizer.transform(test_X)
y = np.ones((len(ppostrain_X)))
z = np.zeros((len(pnegtrain_X)))
y_true = np.concatenate([y, z])
clf = MultinomialNB()
clf.fit(X1, y_true)
y_pred = clf.predict(X2)
print('Accuracy is: ', accuracy_score(y_true, y_pred)) # y_true used as gold labels for both training data and test data since data divided evenly.
print('Confusion matrix is: \n')
print(confusion_matrix(y_true, y_pred))

Accuracy is:  0.82644
Confusion matrix is: 

[[11001  1499]
 [ 2840  9660]]
